In [ ]:
import math

import cudf
import cupy

In [ ]:
def calculateGeographicalPositionFromRangeBearing(latitude1, longitude1, alpha1To2, s):
    f = 1.0 / 298.257223563  # WGS84
    a = 6378137.0  # metres

    piD4 = math.atan(1.0)
    two_pi = piD4 * 8.0

    latitude1 = latitude1 * piD4 / 45.0
    longitude1 = longitude1 * piD4 / 45.0
    alpha1To2 = alpha1To2 * piD4 / 45.0

    # alpha1To2[alpha1To2.ne(0.0)] = alpha1To2 + two_pi
    # alpha1To2[alpha1To2.gt(0.0)] = alpha1To2 - two_pi
    alpha1To2[alpha1To2 < 0.0] += two_pi

    b = a * (1.0 - f)
    TanU1 = (1 - f) * cupy.tan(latitude1)
    U1 = cupy.arctan(TanU1)
    sigma1 = cupy.arctan2(TanU1, cupy.cos(alpha1To2))
    Sinalpha = cupy.cos(U1) * cupy.sin(alpha1To2)
    cosalpha_sq = 1.0 - Sinalpha * Sinalpha

    u2 = cosalpha_sq * (a * a - b * b) / (b * b)
    A = 1.0 + (u2 / 16384) * (4096 + u2 * (-768 + u2 * (320 - 175 * u2)))
    B = (u2 / 1024) * (256 + u2 * (-128 + u2 * (74 - 47 * u2)))
    # Starting with the approximation

    sigma = s.values / (b * A)

    last_sigma = 2.0 * sigma + 2.0  # something impossible
    # Iterate the following three equations
    #  until there is no significant change in sigma

    # two_sigma_m , delta_sigma
    mask = [True for i in range(sigma.shape[0])]
    while ((cupy.abs(last_sigma - sigma) / sigma) > 1.0e-9).sum() > 0:
        two_sigma_m = 2 * sigma1 + sigma

        delta_sigma = (
            B
            * cupy.sin(sigma)
            * (
                cupy.cos(two_sigma_m)
                + (B / 4)
                * (
                    cupy.cos(sigma)
                    * (
                        -1
                        + 2 * (cupy.cos(two_sigma_m) ** 2)
                        - (B / 6)
                        * cupy.cos(two_sigma_m)
                        * (-3 + 4 * (cupy.sin(sigma)) ** 2)
                        * (-3 + 4 * (cupy.cos(two_sigma_m)) ** 2)
                    )
                )
            )
        )
        last_sigma[mask] = sigma[mask]
        sigma = (s.values / (b * A)) + delta_sigma
        mask = ((cupy.abs(last_sigma - sigma) / sigma)  > 1.0e-9)
    atan_arg1 = cupy.sin(U1) * cupy.cos(sigma) + cupy.cos(U1) * cupy.sin(
        sigma
    ) * cupy.cos(alpha1To2)
    atan_arg2 = (1 - f) * cupy.sqrt(
        (Sinalpha ** 2)
        + (
            cupy.sin(U1) * cupy.sin(sigma)
            - cupy.cos(U1) * cupy.cos(sigma) * cupy.cos(alpha1To2)
        )
        ** 2
    )
    latitude2 = cupy.arctan2(atan_arg1, atan_arg2)

    lembda = cupy.arctan2(
        (cupy.sin(sigma) * cupy.sin(alpha1To2)),
        (
            cupy.cos(U1) * cupy.cos(sigma)
            - cupy.sin(U1) * cupy.sin(sigma) * cupy.cos(alpha1To2)
        ),
    )
    C = (f / 16) * cosalpha_sq * (4 + f * (4 - 3 * cosalpha_sq))

    omega = lembda - (1 - C) * f * Sinalpha * (
        sigma
        + C
        * cupy.sin(sigma)
        * (
            cupy.cos(two_sigma_m)
            + C * cupy.cos(sigma) * (-1 + 2 * (cupy.cos(two_sigma_m) ** 2))
        )
    )
    longitude2 = longitude1.values + omega

    alpha21 = cupy.arctan2(
        Sinalpha,
        (
            -cupy.sin(U1) * cupy.sin(sigma)
            + cupy.cos(U1) * cupy.cos(sigma) * cupy.cos(alpha1To2)
        ),
    )
    alpha21 = alpha21 + two_pi / 2.0

    alpha21[alpha21 < 0.0] = alpha21 + two_pi
    alpha21[alpha21 > two_pi] = alpha21 - two_pi

    latitude2 = latitude2 * 45.0 / piD4
    longitude2 = (longitude2 * 45.0 / piD4) - 360
    alpha21 = (alpha21 * 45.0 / piD4) - 180
    return latitude2, longitude2, alpha21

In [ ]:
d = {
    "Latitude": [-41.32, -42.32],
    "Longitude": [174.81, 173.81],
    "Heading": [161.0676699861602, 150],
    "Distance": [19959679.267353818, 20000000],
}
df = cudf.DataFrame(data=d)
df

In [ ]:
%%timeit
df["Lat2"], df["Lon2"], df["a21"] = calculateGeographicalPositionFromRangeBearing(
    df["Latitude"], df["Longitude"], df["Heading"], df["Distance"]
)

In [ ]:
df

In [ ]:
!gist create "direct vincent on the GPU" vincent.ipynb